In [1]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd

# from keras.models import Sequential
# from keras.layers import Dense
from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.model_selection import RepeatedKFold
# from sklearn.metrics import accuracy_score

import soundfile as sf
# from tqdm import tqdm
import os


In [2]:
DEV_ROOT = './Data/FSD50K.dev_audio/'
TEST_ROOT = './Data/FSD50K.eval_audio/'

raw_dev_df = pd.read_csv('./Data/FSD50K.ground_truth/dev.csv', dtype={'fname': 'string'})

test_df = pd.read_csv('./Data/FSD50K.ground_truth/eval.csv', dtype={'fname': 'string'})
vocab_df = pd.read_csv('./Data/FSD50K.ground_truth/vocabulary.csv', index_col=0, header=None, names=['label','mids'])
vocab_dict = dict(zip(vocab_df.mids, vocab_df.label))

In [3]:
raw_dev_df.head()

,fname,labels,mids,split
0,64760,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf",train
1,16399,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf",train
2,16401,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf",train
3,16402,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf",train
4,16404,"Electric_guitar,Guitar,Plucked_string_instrume...","/m/02sgy,/m/0342h,/m/0fx80y,/m/04szw,/m/04rlf",train


In [4]:
raw_dev_df['mids'] = raw_dev_df['mids'].apply(lambda x: tuple(x.split(',')))
test_df['mids'] = test_df['mids'].apply(lambda x: tuple(x.split(',')))

In [5]:
one_hot_encoder = MultiLabelBinarizer()
one_hot_encoder.fit(raw_dev_df['mids'])
output_vec = one_hot_encoder.transform(raw_dev_df['mids'])
raw_dev_df['output_vec'] = output_vec.tolist()
output_vec = one_hot_encoder.transform(test_df['mids'])
test_df['output_vec'] = output_vec.tolist()

In [6]:
len(raw_dev_df.output_vec[0])

200

In [7]:
raw_dev_df

,fname,labels,mids,split,output_vec
0,64760,"Electric_guitar,Guitar,Plucked_string_instrume...","(/m/02sgy, /m/0342h, /m/0fx80y, /m/04szw, /m/0...",train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,16399,"Electric_guitar,Guitar,Plucked_string_instrume...","(/m/02sgy, /m/0342h, /m/0fx80y, /m/04szw, /m/0...",train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,16401,"Electric_guitar,Guitar,Plucked_string_instrume...","(/m/02sgy, /m/0342h, /m/0fx80y, /m/04szw, /m/0...",train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,16402,"Electric_guitar,Guitar,Plucked_string_instrume...","(/m/02sgy, /m/0342h, /m/0fx80y, /m/04szw, /m/0...",train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,16404,"Electric_guitar,Guitar,Plucked_string_instrume...","(/m/02sgy, /m/0342h, /m/0fx80y, /m/04szw, /m/0...",train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
40961,102863,"Fowl,Livestock_and_farm_animals_and_working_an...","(/m/025rv6n, /m/0ch8v, /m/0jbk)",train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
40962,389607,"Fowl,Livestock_and_farm_animals_and_working_an...","(/m/025rv6n, /m/0ch8v, /m/0jbk)",train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
40963,90091,"Fowl,Livestock_and_farm_animals_and_working_an...","(/m/025rv6n, /m/0ch8v, /m/0jbk)",train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
40964,244718,"Fowl,Livestock_and_farm_animals_and_working_an...","(/m/025rv6n, /m/0ch8v, /m/0jbk)",train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [33]:
train_df = raw_dev_df[raw_dev_df['split'] == 'train']
val_df = raw_dev_df[raw_dev_df['split'] == 'val']

In [8]:
dd = DEV_ROOT + '64760' + '.wav'
sound_clip,sr = librosa.load(dd)
sr

22050

In [41]:
def replicate(data, min_clip_len):
  if len(data) < min_clip_len:
    tile_size = (min_clip_len // data.shape[0]) + 1
    data = np.tile(data, tile_size)[:min_clip_len]
  return data

In [65]:
def extract_features(fname_list,output_vec_list,DIR,bands=60,frames=41):
    def _windows(data, window_size):
        start = 0
        while start < len(data):
            yield int(start), int(start + window_size)
            start += (window_size // 2)
            
    window_size = 512 * (frames - 1)
    min_clip_len = int(22050 * 1)
    features, labels = [], []
    for idx,file_name in enumerate(tqdm(fname_list)):
        fn = DIR + file_name + '.wav'
        segment_log_specgrams, segment_labels = [], []
        sound_clip,sr = librosa.load(fn)
        label = output_vec_list[idx]
        if len(sound_clip) < min_clip_len:
            sound_clip = replicate(sound_clip, min_clip_len)
        for (start,end) in _windows(sound_clip,window_size):
            if(len(sound_clip[start:end]) == window_size):
                signal = sound_clip[start:end]
                melspec = librosa.feature.melspectrogram(y=signal,n_mels=bands)
                logspec = librosa.amplitude_to_db(melspec)
                logspec = logspec.T.flatten()[:, np.newaxis].T
                segment_log_specgrams.append(logspec)
                segment_labels.append(label)
            
        segment_log_specgrams = np.asarray(segment_log_specgrams).reshape(
            len(segment_log_specgrams),bands,frames,1)
        segment_features = np.concatenate((segment_log_specgrams, np.zeros(
            np.shape(segment_log_specgrams))), axis=3)
        for i in range(len(segment_features)): 
            segment_features[i, :, :, 1] = librosa.feature.delta(
                segment_features[i, :, :, 0])
        
        if len(segment_features) > 0: # check for empty segments 
            features.append(segment_features)
            labels.append(segment_labels)
    return features, labels

In [43]:
raw_dev_df['fname'][:5].to_list()

['64760', '16399', '16401', '16402', '16404']

In [46]:
train_df = raw_dev_df[raw_dev_df['split'] == 'train']
val_df = raw_dev_df[raw_dev_df['split'] == 'val']

In [64]:
from tqdm import tqdm

In [66]:
features_tr, labels_tr = extract_features(train_df['fname'][:500].to_list(), train_df['output_vec'][:500].to_list(), DEV_ROOT)

 24%|███████████████████▎                                                            | 121/500 [00:15<00:50,  7.57it/s]


KeyboardInterrupt: 

In [48]:
features_val, labels_val = extract_features(val_df['fname'][:500].to_list(), val_df['output_vec'][:500].to_list(), DEV_ROOT)

In [49]:
STORE_DIR = './Data/Checkpoints/'
np.savez_compressed(STORE_DIR+'train', features=np.asarray(features_tr,dtype=object), labels=np.asarray(labels_tr,dtype=object))
np.savez_compressed(STORE_DIR+'val', features=np.asarray(features_val,dtype=object), labels=np.asarray(labels_val,dtype=object))

In [50]:
loaded_tr = np.load(STORE_DIR+'train'+'.npz', allow_pickle=True)
loaded_val = np.load(STORE_DIR+'val'+'.npz', allow_pickle=True)

In [51]:
features_tr = loaded_tr["features"]
labels_tr = loaded_tr["labels"]
features_val = loaded_val["features"]
labels_val = loaded_val["labels"]

In [54]:
features_tr[0].shape

(2, 60, 41, 2)

In [40]:
np.asarray(labels_tr[0]).shape

(11, 200)

In [55]:
import tensorflow as tf
from tensorflow import keras

In [56]:
def get_network():
    num_filters = [24,32,64,128] 
    pool_size = (2, 2) 
    kernel_size = (3, 3)  
    input_shape = (60, 41, 2)
    num_classes = 200
    keras.backend.clear_session()
    
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(24, kernel_size,
                padding="same", input_shape=input_shape))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))

    model.add(keras.layers.Conv2D(32, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))  
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
    
    model.add(keras.layers.Conv2D(64, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))  
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
    
    model.add(keras.layers.Conv2D(128, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))  

    model.add(keras.layers.GlobalMaxPooling2D())
    model.add(keras.layers.Dense(128, activation="relu"))
    model.add(keras.layers.Dense(num_classes, activation="sigmoid"))

    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), 
        loss=keras.losses.BinaryCrossentropy(), 
        metrics=["AUC"])
    return model

In [59]:
# x_tr = np.asarray(features)
# y_tr = np.asarray(labels)

x_tr = np.concatenate(features_tr, axis=0) 
y_tr = np.concatenate(labels_tr, axis=0) 

x_val = np.concatenate(features_val, axis=0) 
y_val= np.concatenate(labels_val, axis=0) 

model = get_network()
model.fit(x_tr, y_tr, validation_data = (x_val,y_val), epochs = 10, batch_size = 24, verbose = 1, shuffle=True)

Epoch 1/10
126/126 [==============================] - 10s 73ms/step - loss: 0.4150 - auc: 0.8150 - val_loss: 0.2956 - val_auc: 0.5846
Epoch 2/10
126/126 [==============================] - 10s 77ms/step - loss: 0.0382 - auc: 0.9925 - val_loss: 0.1240 - val_auc: 0.6148
Epoch 3/10
126/126 [==============================] - 9s 75ms/step - loss: 0.0156 - auc: 0.9968 - val_loss: 0.1401 - val_auc: 0.6266
Epoch 4/10
126/126 [==============================] - 10s 81ms/step - loss: 0.0109 - auc: 0.9986 - val_loss: 0.1543 - val_auc: 0.6214
Epoch 5/10
126/126 [==============================] - 10s 77ms/step - loss: 0.0077 - auc: 0.9992 - val_loss: 0.1599 - val_auc: 0.6111
Epoch 6/10
126/126 [==============================] - 9s 74ms/step - loss: 0.0055 - auc: 0.9995 - val_loss: 0.1589 - val_auc: 0.6150
Epoch 7/10
126/126 [==============================] - 9s 74ms/step - loss: 0.0043 - auc: 0.9996 - val_loss: 0.1657 - val_auc: 0.6183
Epoch 8/10
126/126 [==============================] - 9s 74ms/ste

(720, 60, 41, 2)

In [105]:
y_pred = model.predict(features[0])

In [35]:
temp = np.asarray([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [28]:
from sklearn import metrics

In [62]:
features_tr[0].shape

(2, 60, 41, 2)

In [106]:
# first index in latter element because same copied down the stream
# np.seterr(divide='', invalid='')
metrics.average_precision_score(np.asarray(labels[0])[1],y_pred[1], average=None)

1.0

In [41]:
metrics.average_precision_score(np.asarray(labels[2])[0],y_pred[0], average=None)

1.0

In [ ]:
metrics.average_precision_score(np.asarray(labels[0]),y_pred)

In [ ]:
metrics.average_precision_score(np.asarray(labels[0]),np.where(y_pred<0.5, 0, 1), average=None)

In [ ]:
y_pred

In [151]:
list(y_pred[:,0])

[9.70866e-12, 6.3639215e-12]

In [ ]:
np.asarray(labels[0])

In [166]:
np.asarray(labels[0])[:,48]

array([1, 1])

In [170]:
y_pred[:,48]

array([0.5727001 , 0.56160223], dtype=float32)

In [25]:
np.asarray(labels[0])[:,47]

array([0, 0])

In [26]:
y_pred[:,47]

array([3.8880571e-05, 4.4469878e-05], dtype=float32)

In [188]:
metrics.average_precision_score([0,0],y_pred[:,47])

C:\Users\simma\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


nan

In [38]:
np.where(y_pred<0.5, 0, 1)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 

In [195]:
np.asarray(labels[0]).T.shape

(200, 2)

In [44]:
metrics.average_precision_score(np.asarray(labels[3]).T,np.where(y_pred<0.5, 0, 1).T)

1.0

In [205]:
np.asarray(labels[0]).shape

(2, 200)